In [9]:
import json
import kubernetes
kubernetes.config.load_kube_config()
k8s_api = kubernetes.client.CoreV1Api()
k8s_watch = kubernetes.watch.Watch()

In [15]:
print(kubernetes)

<module 'kubernetes' from 'C:\\Users\\Ark\\AppData\\Local\\Programs\\Python\\Python36\\lib\\site-packages\\kubernetes\\__init__.py'>


In [ ]:
// addPodMetadata is helper to either add a pod label or annotation to the pod
func addPodMetadata(c kubernetes.Interface, field, podName, namespace, key, value string) error {
	metadata := map[string]interface{}{
		"metadata": map[string]interface{}{
			field: map[string]string{
				key: value,
			},
		},
	}
	var err error
	patch, err := json.Marshal(metadata)
	if err != nil {
		return errors.InternalWrapError(err)
	}
	for attempt := 0; attempt < patchRetries; attempt++ {
		_, err = c.CoreV1().Pods(namespace).Patch(podName, types.MergePatchType, patch)
		if err != nil {
			if !apierr.IsConflict(err) {
				return err
			}
		} else {
			break
		}
		time.Sleep(100 * time.Millisecond)
	}
	return err
}

In [18]:
import json
patch_retries = 100
def add_pod_metadata(
    k8s_api: kubernetes.client.CoreV1Api,
    namespace: str,
    pod_name: str,
    field: str,
    key: str,
    value: str,
):
    patch = {
        'metadata': {
            field: {key: value}
        }
    }
    for retry in range(patch_retries):
        try:
            pod = k8s_api.patch_namespaced_pod(
                name=pod_name,
                namespace=namespace,
                body=json.dumps(patch),
            )
            return
        except Exception e:
            print(e)
            # TODO: Sleep for 100ms
    

SyntaxError: invalid syntax (<ipython-input-18-99b5718a55ef>, line 16)

In [ ]:
executions = {} # id -> pod_name
execution_artifacts = {} # id = (execution_id, output_name) -> artifact_id
artifacts = {} # id -> Type, URL

ARGO_OUTPUTS_ANNOTATION_KEY = 'workflows.argoproj.io/outputs'
COMPONENT_SPEC_ANNOTATION_KEY = 'pipelines.kubeflow.org/component_spec'
EXECUTION_ID_ANNOTATION_KEY = 'pipelines.kubeflow.org/metadata_execution_id'
ARTIFACT_IDS_ANNOTATION_KEY = 'pipelines.kubeflow.org/metadata_artifact_ids'

namespace = 'kubeflow' # FIX!!!

# Fixed - TODO: Process existing pods first (but how to catch up the events? Maybe they can be replayed)
#               Check for Argo Pods without "metadata_artifacts_written: True" label

# list_pod_for_all_namespaces:
#   resource_version: Defaults to changes from the beginning of history.
#   label_selector: A selector to restrict the list of returned objects by their labels. Defaults to everything.
#   field_selector: A selector to restrict the list of returned objects by their fields. Defaults to everything.

for event in k8s_watch.stream(k8s_api.list_pod_for_all_namespaces):
    try:
        obj = event['object']
        if obj.kind == 'Pod': # TODO: Detect Argo pods
            if event['type'] == 'ADDED':
                execution_id = str(len(executions) + 1)
                executions[execution_id] = obj.metadata.name
                # !!! Add annotation:
                obj.metadata.annotations[EXECUTION_ID_ANNOTATION_KEY] = execution_id
                add_pod_metadata(
                    namespace=namespace,
                    pod_name=obj.metadata.name,
                    field='annotations',
                    key=EXECUTION_ID_ANNOTATION_KEY,
                    value=execution_id,
                )

            #elif event['object'] == 'MODIFIED':
            if obj.status.phase not in ['Pending', 'Running']: # phase = One of Pending,Running,Succeeded,Failed,Unknown
                assert obj.metadata.name in executions.values() # The pod name should have been added before
                if obj.status.phase == 'Succeeded':
                    if ARGO_OUTPUTS_ANNOTATION_KEY in obj.metadata.annotations: # Should be present
                        argo_outputs = json.loads(obj.metadata.annotations[ARGO_OUTPUTS_ANNOTATION_KEY])
                        component_spec = json.loads(obj.metadata.annotations[COMPONENT_SPEC_ANNOTATION_KEY])
                        
                        for artifact in argo_outputs.get('artifacts', []):
                            art_name = artifact['name']
                            art_key = artifact['key']

                if obj.status.phase == 'Failed':
                    pass

        if kind == 'Pod' and name == pod_name:
            pod_events.append(event)
            phase = obj.status.phase #One of Pending,Running,Succeeded,Failed,Unknown
            print(\"Event: %s %s %s %s\" % (event_type, kind, name, phase)) 
            if phase != 'Pending':
                k8s_watch.stop()
    except Exception e:
        print(event)
        print(e)